In [ ]:
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import mercury as mr

In [ ]:
#CONTOURS COLOUR (RGB)
R_color = 0
G_color = 255
B_color = 154

#GAUSSIAN BLUR COEFFICIENT
GC1 = GC2 = 1

#TASSELED CAP COEFFICIENTS
coefficients = [[0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863],
                [-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800],
                [0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572],
                [-0.8242, 0.0849, 0.4393, -0.0580, 0.2012, -0.2774],
                [-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085],
                [0.1084, -0.9022, 0.4119, -0.0575, 0.0029, -0.0251]]

#HIGH PASS FILTER KERNEL
hp_kernel = np.array([[0.0, -1.0, 0.0],
                      [-1.0, 4.0, -1.0],
                      [0.0, -1.0, 0.0]])

#SEPIA FILTER CONFIG
sepia_kernel = np.array([[0.272, 0.534, 0.131],[0.349, 0.686, 0.168],[0.393, 0.769, 0.189]])
ddepth = -1

#PANCHROMATIC FILTER KERNEL
panchromatic_filter = np.array([[0.5, 1, 0.5],
                                [1, 1, 1],
                                [0.5, 1, 0.5]])

#CLAHE FILTER CONFIG
clipLimit = 5.0
tileGridSize = (8, 8)

#COMPONENT CONNECTIVITY CONFIG
thresh = 100
maxval = 255
thickness = 2
connectivity = 8

#TRANSMISSION FILTER KERNEL CONFIG
kernel_size = 15

In [ ]:
app = mr.App(title="GAIA v0.2b [BETA]",
             description="Open-Source Geospatial & Aerial Analysis",
             show_code=False,
             show_prompt=False,
             continuous_update=True,
             static_notebook=False,
             show_sidebar=True,
             full_screen=True,
             allow_download=True)

mpl.rcParams['figure.dpi'] = 400

In [ ]:
mr.Md('''

Welcome to GAIA - Geospatial & Aerial Images Analyser [BETA]

[!] This program solution is not final and some details are potentially to be changed [!]\n
[!] New filters may be added to this program soon or later [!]\n
[!] Current version - 0.2b [!]

There's only 2 steps to start:\n
1) Upload an image using the context menu on the left\n
2) Select filters from the menu on the left\n
Then you get the result. You can select multiple filters or cancel your selection and everything will be auto-updated\n
''')
uploaded = mr.File(label="File upload", max_file_size="10MB")
sidebar_filters = mr.Note(text=" "*25 + "FILTERS LIST")
print(f"Uploaded file name: {uploaded.filename}")
print(f"Uploaded file path: {uploaded.filepath}")

image = cv2.imread(uploaded.filepath)

if image is not None:
    plt.imshow(image)
    plt.title('UPLOADED IMAGE')
    plt.show()

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

    new_image = np.zeros(image.shape, dtype='uint8')
    image_gray_blurred = cv2.GaussianBlur(image_gray, (GC1, GC2), 0)
    image_gray_blurred_edges = cv2.Canny(image_gray_blurred, 100, 140)

    contours, hierarchy = cv2.findContours(image_gray_blurred_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours, hir = cv2.findContours(image_gray_blurred_edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    image_contoured = cv2.drawContours(new_image, contours, -1, (R_color, G_color, B_color), 1)

    kernel = hp_kernel/(np.sum(hp_kernel) if np.sum(hp_kernel)!=0 else 1)
    image_hp = cv2.filter2D(image,-1,kernel)

    image_sepia = cv2.filter2D(src=image, kernel=sepia_kernel, ddepth=ddepth)

    image_denoised = cv2.fastNlMeansDenoisingColored(image,None,20,10,7,21)

    image_red = image.copy()
    image_red[:, :, 0] = 0
    image_red[:, :, 1] = 0

    image_green = image.copy()
    image_green[:, :, 0] = 0
    image_green[:, :, 2] = 0

    image_blue = image.copy()
    image_blue[:, :, 1] = 0
    image_blue[:, :, 2] = 0

    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    image_clahe = clahe.apply(image_gray)

    image_ccp = image.copy()
    _, binarized_img = cv2.threshold(image_gray, thresh=thresh, maxval=maxval, type=cv2.THRESH_BINARY)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binarized_img, connectivity=connectivity)

    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        rect_color = (0, 255, 0)
        image_cc = cv2.rectangle(image_ccp, (x, y), (x + w, y + h), rect_color, thickness=thickness)

    image_inv_sat = image_hsv.copy()
    image_inv_sat[:, :, 1] = 255 - image_inv_sat[:, :, 1]
    image_inv_sat = cv2.cvtColor(image_inv_sat, cv2.COLOR_HSV2BGR)

    image_sat = image_hsv.copy()
    image_sat[:, :, 0] = 0
    image_sat[:, :, 2] = 255
    image_sat = cv2.cvtColor(image_sat, cv2.COLOR_HSV2BGR)

    image_hue = image_hsv.copy()
    image_hue[:, :, 1] = 255
    image_hue[:, :, 2] = 255
    image_hue = cv2.cvtColor(image_hue, cv2.COLOR_HSV2BGR)

    rgb_img = image.copy()
    rgb_img[:, :, 0] = cv2.equalizeHist(image[:, :, 0])
    rgb_img[:, :, 1] = cv2.equalizeHist(image[:, :, 1])
    rgb_img[:, :, 2] = cv2.equalizeHist(image[:, :, 2])

    img_cool = cv2.applyColorMap(image, cv2.COLORMAP_COOL)

    to_spectral = image_rgb.copy()
    image_lwir = to_spectral[:, :, 2]

    image_swir = to_spectral[:, :, 1]

    to_ndvi = image_rgb.copy()
    r, g, b = cv2.split(to_ndvi)
    sum_img = g + r
    mask = sum_img == 0
    image_ndvi = np.zeros_like(g, dtype=np.float32)
    image_ndvi[~mask] = (g - r)[~mask] / sum_img[~mask]

    red = image[:, :, 0]
    green = image[:, :, 1]
    blue = image[:, :, 2]

    tc_1 = coefficients[1][0] * blue + coefficients[1][1] * green + coefficients[1][2] * red + \
           coefficients[1][3] * 0 + coefficients[1][4] * 0 + coefficients[1][5] * 0

    tc_2 = coefficients[2][0] * blue + coefficients[2][1] * green + coefficients[2][2] * red + \
           coefficients[2][3] * 0 + coefficients[2][4] * 0 + coefficients[2][5] * 0

    tc_3 = coefficients[4][0] * blue + coefficients[4][1] * green + coefficients[4][2] * red + \
           coefficients[4][3] * 0 + coefficients[4][4] * 0 + coefficients[4][5] * 0

    tc_4 = coefficients[5][0] * blue + coefficients[5][1] * green + coefficients[5][2] * red + \
           coefficients[5][3] * 0 + coefficients[5][4] * 0 + coefficients[5][5] * 0

    tc_1_norm = (tc_1 - tc_1.min()) / (tc_1.max() - tc_1.min())
    tc_2_norm = (tc_2 - tc_2.min()) / (tc_2.max() - tc_2.min())
    tc_3_norm = (tc_3 - tc_3.min()) / (tc_3.max() - tc_3.min())
    tc_4_norm = (tc_4 - tc_4.min()) / (tc_4.max() - tc_4.min())

    atmospheric_light = np.percentile(image, 99)
    transmission = 1 - 0.95 * (image.astype(np.float32) / atmospheric_light)

    if image.dtype != np.uint8:
        image = image.astype(np.uint8)
    if image.ndim == 3 and image.shape[-1] not in [1, 3, 4]:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    transmission = (cv2.medianBlur((255*transmission).astype(np.uint8), kernel_size)).astype(np.float32) / 255

    filtered_img = cv2.filter2D(image, -1, panchromatic_filter)

    image_inv = cv2.bitwise_not(image)

    image_laplacian = cv2.Laplacian(image_gray, cv2.CV_64F)
    image_laplacian = np.uint8(np.absolute(image_laplacian))

    sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5)
    sobely = cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5)
    sobel = sobelx + sobely

In [ ]:
def bgr_filter(image):
    plt.imshow(image)
    plt.title('BGR FILTER')
    plt.show()

In [ ]:
def gray_filter(image_gray):
    plt.imshow(image_gray)
    plt.title('GRAY FILTER')
    plt.show()

In [ ]:
def hsv_filter(image_hsv):
    plt.imshow(image_hsv)
    plt.title('HSV FILTER')
    plt.show()

In [ ]:
def contour_image(image_contoured):
    plt.imshow(image_contoured)
    plt.title('CONTOURING')
    plt.show()

In [ ]:
def hp_image(image_hp):
    plt.imshow(image_hp)
    plt.title('HIGH-PASS FILTER')
    plt.show()

In [ ]:
def sepia_image(image_sepia):
    plt.imshow(image_sepia, cmap="gray")
    plt.title('SEPIA FILTER')
    plt.show()

In [ ]:
def denoised_image(image_denoised):
    plt.imshow(image_denoised)
    plt.title('DENOISING FILTER')
    plt.show()

In [ ]:
def red_image(image_red):
    plt.imshow(cv2.cvtColor(image_red, cv2.COLOR_BGR2RGB))
    plt.title('RED FILTER')
    plt.show()

In [ ]:
def green_image(image_green):
    plt.imshow(cv2.cvtColor(image_green, cv2.COLOR_BGR2RGB))
    plt.title('GREEN FILTER')
    plt.show()

In [ ]:
def blue_image(image_blue):
    plt.imshow(cv2.cvtColor(image_blue, cv2.COLOR_BGR2RGB))
    plt.title('BLUE FILTER')
    plt.show()

In [ ]:
def sat_image(image_sat):
    plt.imshow(cv2.cvtColor(image_sat, cv2.COLOR_BGR2RGB))
    plt.title('SATURATION FILTER')
    plt.show()

In [ ]:
def hue_image(image_hue):
    plt.imshow(cv2.cvtColor(image_hue, cv2.COLOR_BGR2RGB))
    plt.title('HUE FILTER')
    plt.show()

In [ ]:
def inv_image(image_inv):
    plt.imshow(cv2.cvtColor(image_inv, cv2.COLOR_BGR2RGB))
    plt.title('INVERSION FILTER')
    plt.show()

In [ ]:
def img_rgb(rgb_img):
    plt.imshow(cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB))
    plt.title('RGB FILTER')
    plt.show()

In [ ]:
def inv_sat_img(image_inv_sat):
    plt.imshow(cv2.cvtColor(image_inv_sat, cv2.COLOR_BGR2RGB))
    plt.title('SATURATION INVERSION FILTER')
    plt.show()

In [ ]:
def clahe_image(image_clahe):
    plt.imshow(cv2.cvtColor(image_clahe, cv2.COLOR_GRAY2RGB))
    plt.title('CLAHE FILTER')
    plt.show()

In [ ]:
def cc_image(image_cc):
    plt.imshow(cv2.cvtColor(image_cc, cv2.COLOR_BGR2RGB))
    plt.title('COMPONENT CONNECTIVITY FILTER (ORIGINAL IMAGE)')
    plt.show()

In [ ]:
def cc_black(labels):
    plt.imshow(labels, cmap='jet')
    plt.title('COMPONENT CONNECTIVITY FILTER (MONOTONOUS BACKGROUND)')
    plt.show()

In [ ]:
def cool_img(img_cool):
    plt.imshow(img_cool)
    plt.title('COOL FILTER')
    plt.show()

In [ ]:
def ndvi_img(image_ndvi):
    plt.imshow(image_ndvi, cmap='jet')
    plt.title('NDVI FILTER')
    plt.show()

In [ ]:
def swir_image(image_swir):
    plt.imshow(image_swir) #cmap='gray'
    plt.title('SWIR FILTER')
    plt.show()

In [ ]:
def lwir_image(image_lwir):
    plt.imshow(image_lwir)  #cmap gray
    plt.title('LWIR FILTER')
    plt.show()

In [ ]:
def filt_img(filtered_img):
    plt.imshow(filtered_img)
    plt.title('PANCHROMATIC FILTER')
    plt.show()

In [ ]:
def transmission(transmission):
    plt.imshow(transmission)
    plt.title('TRANSMISSION FILTER')
    plt.show()

In [ ]:
def tc_1(tc_1_norm):
    plt.imshow(tc_1_norm)
    plt.title('TASSELED CAP FILTER (1)')
    plt.show()

In [ ]:
def tc_2(tc_2_norm):
    plt.imshow(tc_2_norm)
    plt.title('TASSELED CAP FILTER (2)')
    plt.show()

In [ ]:
def tc_3(tc_3_norm):
    plt.imshow(tc_3_norm)
    plt.title('TASSELED CAP FILTER (3)')
    plt.show()

In [ ]:
def tc_4(tc_4_norm):
    plt.imshow(tc_4_norm)
    plt.title('TASSELED CAP FILTER (4)')
    plt.show()

In [ ]:
def laplacian_image(image_laplacian):
    plt.imshow(image_laplacian)
    plt.title('LAPLACIAN FILTER')
    plt.show()

In [ ]:
def adjusted_image(adjusted_image):
    plt.imshow(adjusted_image)
    plt.title('AUTO-CONTRAST')
    plt.show()

In [ ]:
def sobel(sobel):
    plt.imshow(sobel)
    plt.title("SOBEL FILTER")
    plt.show()

In [ ]:
show_bgr = mr.Checkbox(value=False, label="BGR filter")

if show_bgr.value:
    bgr_filter(image)

In [ ]:
show_hsv = mr.Checkbox(value=False, label="HSV filter")

if show_hsv.value:
    hsv_filter(image_hsv)

In [ ]:
show_hue = mr.Checkbox(value=False, label="HUE filter")

if show_hue.value:
    hue_image(image_hue)

In [ ]:
show_gray = mr.Checkbox(value=False, label="Gray filter")

if show_gray.value:
    gray_filter(image_gray)

In [ ]:
show_cont = mr.Checkbox(value=False, label="Contouring")

if show_cont.value:
    contour_image(image_contoured)

In [ ]:
show_hp = mr.Checkbox(value=False, label="High-pass filter")

if show_hp.value:
    hp_image(image_hp)

In [ ]:
show_sep = mr.Checkbox(value=False, label="Sepia filter")

if show_sep.value:
    sepia_image(image_sepia)

In [ ]:
show_denoise = mr.Checkbox(value=False, label="Denoising")

if show_denoise.value:
    denoised_image(image_denoised)

In [ ]:
show_RGBsp = mr.Checkbox(value=False, label="R, G, B")

if show_RGBsp.value:
    red_image(image_red)
    green_image(image_green)
    blue_image(image_blue)

In [ ]:
show_satur = mr.Checkbox(value=False, label="Saturation filter")

if show_satur.value:
    sat_image(image_sat)

In [ ]:
show_invers = mr.Checkbox(value=False, label="Inversion filter")

if show_invers.value:
    inv_image(image_inv)

In [ ]:
show_RGB = mr.Checkbox(value=False, label="RGB filter")

if show_RGB.value:
    img_rgb(rgb_img)

In [ ]:
show_satinv = mr.Checkbox(value=False, label="Saturation inversion filter")

if show_satinv.value:
    inv_sat_img(image_inv_sat)

In [ ]:
show_clahe = mr.Checkbox(value=False, label="Clahe filter")

if show_clahe.value:
    clahe_image(image_clahe)

In [ ]:
show_ccimage = mr.Checkbox(value=False, label="Component connectivity filter (on image)")

if show_ccimage.value:
    cc_image(image_cc)

In [ ]:
show_ccblack = mr.Checkbox(value=False, label="Component connectivity filter (on black screen)")

if show_ccblack.value:
    cc_black(labels)

In [ ]:
show_cool = mr.Checkbox(value=False, label="Cool filter")

if show_cool.value:
    cool_img(img_cool)

In [ ]:
show_ndvi = mr.Checkbox(value=False, label="NDVI filter")

if show_ndvi.value:
    ndvi_img(image_ndvi)

In [ ]:
show_swir = mr.Checkbox(value=False, label="SWIR filter")

if show_swir.value:
    swir_image(image_swir)

In [ ]:
show_lwir = mr.Checkbox(value=False, label="LWIR filter")

if show_lwir.value:
    lwir_image(image_lwir)

In [ ]:
show_panchrom = mr.Checkbox(value=False, label="Filtering")

if show_panchrom.value:
    filt_img(filtered_img)

In [ ]:
show_trans = mr.Checkbox(value=False, label="Transmission filter")

if show_trans.value:
    transmission(transmission)

In [ ]:
show_tc = mr.Checkbox(value=False, label="Tasseled cap filters")

if show_tc.value:
    tc_1(tc_1_norm)
    tc_2(tc_2_norm)
    tc_3(tc_3_norm)
    tc_4(tc_4_norm)

In [ ]:
show_lapl = mr.Checkbox(value=False, label="Laplacian filter")

if show_lapl.value:
    laplacian_image(image_laplacian)

In [ ]:
show_sob = mr.Checkbox(value=False, label="Sobel filter")

if show_sob.value:
    sobel(sobel)